In [25]:
import json
import requests
from config import fed_api_key
from pprint import pprint
import pandas as pd
import numpy as np

In [26]:
pop = ['AUSPOP','HTNPOP','DFWPOP','SATPOP','ELPPOP','MCLPOP']
hpi = ['ATNHPIUS12420Q','ATNHPIUS26420Q','ATNHPIUS19124Q','ATNHPIUS41700Q','ATNHPIUS21340Q','ATNHPIUS32580Q']

In [27]:
#'https://api.stlouisfed.org/fred/series/observations?series_id=WPU0811&api_key=0a60bfbafed5ab7080d559305d53573a&file_type=json'
base_url='https://api.stlouisfed.org/fred/series/observations?series_id='
mid_url = '&api_key='
end_url='&file_type=json'
urls =[]
dummies = []
for x in pop:
    url = base_url + x + mid_url + fed_api_key + end_url
    urls.append(url)

In [28]:
#pull populations and create CSVs
x = 0
dates = []
values = []
msa = []
for url in urls:
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
        dates.append(data['date'])
        values.append(data['value'])
        msa.append(pop[x])
    x = x+1
df = pd.DataFrame({
    "MetroArea": msa,
    "date": dates,
    "values": values
})
df.to_csv('Resources/Population.csv')

In [29]:
#Pull Pricing index and create CSVs
urls =[]
for y in hpi:
    url = base_url + y + mid_url + fed_api_key + end_url
    urls.append(url)
x = 0
dates = []
values = []
msa = []
for url in urls:
    response = requests.get(url)
    data_json = response.json()
    for data in data_json['observations']:
        dates.append(data['date'])
        values.append(data['value'])
        msa.append(pop[x])
    x = x+1
df = pd.DataFrame({
    "MetroArea": msa,
    "date": dates,
    "values": values
})
df.to_csv('Resources/HPI.csv')

In [34]:
file = "Resources/ZilloHomeValueIndexZip.csv"
zhvi_df = pd.read_csv(file)
zhvi_df = zhvi_df[zhvi_df['State']=='TX']
zhvi_df = zhvi_df.drop(columns=['SizeRank','CountyName','RegionID','RegionType','StateName','Metro','State'])

In [37]:
zhvi2_df = pd.melt(zhvi_df,id_vars=['RegionName','Metro','City'],var_name='Date', value_name='Value')
zhvi2_df

,RegionName,Metro,City,Date,Value
0,77494,Houston-The Woodlands-Sugar Land,Katy,1996-01-31,201649.0
1,77449,Houston-The Woodlands-Sugar Land,Katy,1996-01-31,98464.0
2,77084,Houston-The Woodlands-Sugar Land,Houston,1996-01-31,98773.0
3,79936,El Paso,El Paso,1996-01-31,82995.0
4,78660,Austin-Round Rock,Pflugerville,1996-01-31,148012.0
...,...,...,...,...,...
559349,79326,NaN,Fieldton,2021-07-31,145374.0
559350,77428,Bay City,Palacios,2021-07-31,125276.0
559351,3575,NaN,Clifton,2021-07-31,601214.0
559352,77710,Beaumont-Port Arthur,Beaumont,2021-07-31,79598.0


In [38]:
zhvi2_df.to_csv('Resources/zhvi_zip.csv')